In [1]:
import pandas as pd
import os 
import pathlib 
import glob
from datetime import datetime
from tqdm import tqdm

from dotenv import load_dotenv
from elasticsearch import Elasticsearch

import minio
from minio.error import S3Error
from minio.commonconfig import ENABLED
from minio.versioningconfig import VersioningConfig

from RDSBucket_class import *
from data_profiles import *

import warnings
warnings.filterwarnings("ignore")

#####----------------------------------------------------------------#####
##### preprocessing metadata for bam file
#####----------------------------------------------------------------#####

path_to_save_prep_metadata = "/Users/hieunguyen/src/DVC_system/examples/dummy_from_real/prep_metadata"
os.system("mkdir -p {}".format(path_to_save_prep_metadata))

path_to_main_input = "./examples/dummy_data"    
minio_credential = "credentials.mb.json"
es_credential = "es_credential.json"

donwloaddir = "./examples/download"
os.system("mkdir  -p {}".format(donwloaddir))


es = ESearch(es_credential = es_credential)

search_query = {
  "query": {
    "match": {
      "pipeline": "ECD_WGS_hg19"
    }
  }
}

response = es.es.search(index="wgsbam", body=search_query, size = 1000)

search_resdf = pd.DataFrame([doc['_source'] for doc in response['hits']['hits']])

In [2]:
search_resdf

,Labcode,SequencingID,FileName,FileType,Date,pipeline,project,sub_project,ref_genome,depth,cancer_label,bucket,versionID
0,ZLBE113NB,ZLBE113NB,9-ZLBE113NB_S95053-S97053.sorted.bam,bam,2024-07-12,ECD_WGS_hg19,ECD,ECD_WGS,hg19,low,Pancreatic,wgsbam,e0fc3f33-6421-40c7-a5a6-7bfc590d037d
1,ZMG138NB,ZMG138NB,16-ZMG138NB_S95095-S97095.sorted.bam,bam,2024-07-12,ECD_WGS_hg19,ECD,ECD_WGS,hg19,low,Gastric,wgsbam,95f8ce41-ea4e-4291-8e2a-bbcec46401cb
2,ZK0AAAB71NB,ZK0AAAB71NB,8-ZK0AAAB71NB_S95064-S97064.sorted.bam,bam,2024-07-12,ECD_WGS_hg19,ECD,ECD_WGS,hg19,low,Control,wgsbam,ebb4ff62-1bfd-4236-987a-57976af7e657
3,ZYCAA76NB,ZYCAA76NB,42-ZYCAA76NB_S95012-S97012.sorted.bam,bam,2024-07-12,ECD_WGS_hg19,ECD,ECD_WGS,hg19,low,Cervical,wgsbam,db837dc2-e4ed-4d21-9f6b-8ac1155f1cd3
4,ZK0AAAF43NB,ZK0AAAF43NB,38-ZK0AAAF43NB_S95086-S97086.sorted.bam,bam,2024-07-12,ECD_WGS_hg19,ECD,ECD_WGS,hg19,low,Control,wgsbam,88bd2823-177f-49fa-8275-7bd6f0c79312
...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,ZMB133NB,ZMB133NB,32-ZMB133NB_S95024-S97024.sorted.bam,bam,2024-07-12,ECD_WGS_hg19,ECD,ECD_WGS,hg19,low,Breast,wgsbam,cc4238de-8983-442d-9110-a3f05076e9ea
545,ZMB164NB,ZMB164NB,13-ZMB164NB_S95055-S97055.sorted.bam,bam,2024-07-12,ECD_WGS_hg19,ECD,ECD_WGS,hg19,low,Breast,wgsbam,ced64654-7fbf-4c6b-a6d3-6b7def015c69
546,ZLBE301NB,ZLBE301NB,9-ZLBE301NB_S95017-S97017.sorted.bam,bam,2024-07-12,ECD_WGS_hg19,ECD,ECD_WGS,hg19,low,Head and Neck,wgsbam,5a6cbf30-4a84-4aad-ba4f-d24a6a85d590
547,ZL12859NB,ZL12859NB,11-ZL12859NB_S95053-S97053.sorted.bam,bam,2024-07-12,ECD_WGS_hg19,ECD,ECD_WGS,hg19,low,Lung,wgsbam,e6de6d79-9746-4c64-a225-bb0dca084b0f
